In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
mg = RasterModelGrid((50,75),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
#d_h = mg.add_ones('link', 'hydraulic__diameter')

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)

h = mg.add_zeros('node', 'hydraulic__head')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][1521]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 1.*0.1
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print "Number of links = ", mg.number_of_links
print "Number of nodes = ", mg.number_of_nodes
print "Number of active links = ", mg.number_of_active_links
print "Number of core nodes = ", mg.number_of_core_nodes

#code added for fsolve algorithm
dhdx = mg.add_zeros('link', 'hydraulic_head_gradient', noclobber=False)
net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)


pfn = PresFlowNetwork(mg)
pfn.run_one_step()
mc = MeltCreep(mg, dt=500.)

Number of links =  7375
Number of nodes =  3750
Number of active links =  6935
Number of core nodes =  3504


/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/pressurized_flow.py:145: RuntimeWarning: divide by zero encountered in divide
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)


Number of iterations = 1 tolerance = 0.500868296748
Number of iterations = 2 tolerance = 0.345905800073
Number of iterations = 3 tolerance = 0.199129400006
Number of iterations = 4 tolerance = 0.132391947025
Number of iterations = 5 tolerance = 0.0944431742387
Number of iterations = 6 tolerance = 0.0718622352342
Number of iterations = 7 tolerance = 0.0339549429388
Number of iterations = 8 tolerance = 0.0389570746104
Number of iterations = 9 tolerance = 0.0224643940083
Number of iterations = 10 tolerance = 0.0108932530204
Number of iterations = 11 tolerance = 0.00543736136203
Number of iterations = 12 tolerance = 0.00353760592258
Number of iterations = 13 tolerance = 0.00176713125139
Number of iterations = 14 tolerance = 0.00115561351005
Number of iterations = 15 tolerance = 0.000899166820328
Number of iterations = 16 tolerance = 0.000449480883114
Number of iterations = 17 tolerance = 0.000230308127556
Number of iterations = 18 tolerance = 0.000114558730987
Number of iterations = 19 tol

In [3]:
def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='jet', magnitude= False, lw=2):
    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    for i, value in enumerate(values):
        xs = [link_head_x[i],link_tail_x[i]]
        ys = [link_head_y[i],link_tail_y[i]]
        img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
    colorbar(scalarMap)


In [ ]:
nsteps = 50000
step_start = 15001
fig1, axs = subplots(2,2,figsize=(9,6))
every = 50
for step in arange(nsteps)+step_start:
    print "step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean()
    pfn.run_one_step()    
    #mg.calc_grad_at_link(h, out=dhdx)
    #h = fsolve(network_residuals, h[mg.core_nodes])
    #Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', magnitude=True)#, vmin=0., vmax=1.m autoscale=False)
        #plot(mg.at_link['conduit__discharge'][mg.active_links])
        #ylabel('')
        #ylim([0,1.5])
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter')#, autoscale=False, vmin=0, vmax=1.)
        subplot(2,2,4)
        plot(mg.at_link['hydraulic__diameter'][mg.active_links])
        #ylim([0.5,1.5])
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        tight_layout()
        savefig(image_name)
        fig1.clf()

#im_ani = animation.ArtistAnimation(fig1,ims)

step = 15001  avg d_h= 0.135493214641
Number of iterations = 1 tolerance = 1.58439547233e-05
Number of iterations = 2 tolerance = 2.54499880326e-09
mean ddh =  -3.15786894911e-06
step = 15002  avg d_h= 0.135490245174
Number of iterations = 1 tolerance = 1.5844551165e-05
Number of iterations = 2 tolerance = 2.37725810932e-09
mean ddh =  -3.15779655043e-06
step = 15003  avg d_h= 0.135487275774
Number of iterations = 1 tolerance = 1.58451568521e-05
Number of iterations = 2 tolerance = 2.22687595643e-09
mean ddh =  -3.15772417213e-06
step = 15004  avg d_h= 0.135484306443
Number of iterations = 1 tolerance = 1.58457497406e-05
Number of iterations = 2 tolerance = 2.09136318041e-09
mean ddh =  -3.15765181514e-06
step = 15005  avg d_h= 0.13548133718
Number of iterations = 1 tolerance = 1.58463497468e-05
Number of iterations = 2 tolerance = 1.96890349609e-09
mean ddh =  -3.15757948139e-06
step = 15006  avg d_h= 0.135478367985
Number of iterations = 1 tolerance = 1.58469523244e-05
Number of iter

Number of iterations = 1 tolerance = 1.58711982043e-05
Number of iterations = 2 tolerance = 4.36097047913e-10
mean ddh =  -3.15456056354e-06
step = 15048  avg d_h= 0.135353722949
Number of iterations = 1 tolerance = 1.58717906565e-05
Number of iterations = 2 tolerance = 4.35521480818e-10
mean ddh =  -3.15448913849e-06
step = 15049  avg d_h= 0.13535075666
Number of iterations = 1 tolerance = 1.58723757125e-05
Number of iterations = 2 tolerance = 4.38302371766e-10
mean ddh =  -3.15441773677e-06
step = 15050  avg d_h= 0.135347790438
Number of iterations = 1 tolerance = 1.58729698141e-05
Number of iterations = 2 tolerance = 4.41285225253e-10
mean ddh =  -3.15434635415e-06
step = 15051  avg d_h= 0.135344824283
Number of iterations = 1 tolerance = 1.5873552986e-05
Number of iterations = 2 tolerance = 4.44218767266e-10
mean ddh =  -3.15427499424e-06
step = 15052  avg d_h= 0.135341858196
Number of iterations = 1 tolerance = 1.58741453863e-05
Number of iterations = 2 tolerance = 4.47092339503e-

Number of iterations = 1 tolerance = 1.58980469966e-05
Number of iterations = 2 tolerance = 6.02062514135e-10
mean ddh =  -3.15129688385e-06
step = 15094  avg d_h= 0.135217342842
Number of iterations = 1 tolerance = 1.58985830982e-05
Number of iterations = 2 tolerance = 6.06981794193e-10
mean ddh =  -3.15122642345e-06
step = 15095  avg d_h= 0.135214379621
Number of iterations = 1 tolerance = 1.58991091537e-05
Number of iterations = 2 tolerance = 6.1183830154e-10
mean ddh =  -3.1511559843e-06
step = 15096  avg d_h= 0.135211416466
Number of iterations = 1 tolerance = 1.58996480851e-05
Number of iterations = 2 tolerance = 6.16752069554e-10
mean ddh =  -3.15108556501e-06
step = 15097  avg d_h= 0.135208453377
Number of iterations = 1 tolerance = 1.59001821889e-05
Number of iterations = 2 tolerance = 6.21855859897e-10
mean ddh =  -3.1510151641e-06
step = 15098  avg d_h= 0.135205490355
Number of iterations = 1 tolerance = 1.59007060433e-05
Number of iterations = 2 tolerance = 6.26878599269e-1

Number of iterations = 1 tolerance = 1.59224122136e-05
Number of iterations = 2 tolerance = 9.14488222246e-10
mean ddh =  -3.14807641962e-06
step = 15140  avg d_h= 0.135081102943
Number of iterations = 1 tolerance = 1.59229450601e-05
Number of iterations = 2 tolerance = 9.24276790619e-10
mean ddh =  -3.1480068763e-06
step = 15141  avg d_h= 0.13507814275
Number of iterations = 1 tolerance = 1.59234681062e-05
Number of iterations = 2 tolerance = 9.338781655e-10
mean ddh =  -3.14793735468e-06
step = 15142  avg d_h= 0.135075182622
Number of iterations = 1 tolerance = 1.5923993446e-05
Number of iterations = 2 tolerance = 9.4389157002e-10
mean ddh =  -3.14786785152e-06
step = 15143  avg d_h= 0.135072222559
Number of iterations = 1 tolerance = 1.59245160312e-05
Number of iterations = 2 tolerance = 9.53992977954e-10
mean ddh =  -3.14779836736e-06
step = 15144  avg d_h= 0.135069262561
Number of iterations = 1 tolerance = 1.59250399239e-05
Number of iterations = 2 tolerance = 9.6422637663e-10
me

Number of iterations = 1 tolerance = 1.59465949928e-05
Number of iterations = 2 tolerance = 1.61652348215e-09
mean ddh =  -3.14489792904e-06
step = 15186  avg d_h= 0.134945001419
Number of iterations = 1 tolerance = 1.59471498635e-05
Number of iterations = 2 tolerance = 1.640578679e-09
mean ddh =  -3.14482929977e-06
step = 15187  avg d_h= 0.134942044214
Number of iterations = 1 tolerance = 1.59476804511e-05
Number of iterations = 2 tolerance = 1.66583877338e-09
mean ddh =  -3.14476067832e-06
step = 15188  avg d_h= 0.134939087073
Number of iterations = 1 tolerance = 1.59481686292e-05
Number of iterations = 2 tolerance = 1.69152580885e-09
mean ddh =  -3.1446920814e-06
step = 15189  avg d_h= 0.134936129996
Number of iterations = 1 tolerance = 1.59486954254e-05
Number of iterations = 2 tolerance = 1.71772615066e-09
mean ddh =  -3.1446235091e-06
step = 15190  avg d_h= 0.134933172984
Number of iterations = 1 tolerance = 1.59492396613e-05
Number of iterations = 2 tolerance = 1.74486852952e-09

...

Number of iterations = 1 tolerance = 2.37377504517e-05
Number of iterations = 2 tolerance = 8.67042770913e-10
mean ddh =  -2.72244790309e-06
step = 32528  avg d_h= 0.0878957532212
Number of iterations = 1 tolerance = 2.37386815798e-05
Number of iterations = 2 tolerance = 8.59753185312e-10
mean ddh =  -2.72241915279e-06
step = 32529  avg d_h= 0.0878931932244
Number of iterations = 1 tolerance = 2.37396048504e-05
Number of iterations = 2 tolerance = 8.62228843879e-10
mean ddh =  -2.72239039914e-06
step = 32530  avg d_h= 0.0878906332545
Number of iterations = 1 tolerance = 2.37405247015e-05
Number of iterations = 2 tolerance = 8.66469812566e-10
mean ddh =  -2.72236164246e-06
step = 32531  avg d_h= 0.0878880733118
Number of iterations = 1 tolerance = 2.37414542056e-05
Number of iterations = 2 tolerance = 8.50359699323e-10
mean ddh =  -2.72233288355e-06
step = 32532  avg d_h= 0.087885513396
Number of iterations = 1 tolerance = 2.37423802328e-05
Number of iterations = 2 tolerance = 8.6255153

Number of iterations = 1 tolerance = 2.37802244978e-05
Number of iterations = 2 tolerance = 8.37390268238e-10
mean ddh =  -2.72112234092e-06
step = 32574  avg d_h= 0.0877780213929
Number of iterations = 1 tolerance = 2.37811536499e-05
Number of iterations = 2 tolerance = 8.65300969766e-10
mean ddh =  -2.72109345982e-06
step = 32575  avg d_h= 0.0877754626426
Number of iterations = 1 tolerance = 2.37820426286e-05
Number of iterations = 2 tolerance = 8.84834899795e-10
mean ddh =  -2.72106457952e-06
step = 32576  avg d_h= 0.0877729039195
Number of iterations = 1 tolerance = 2.37829732609e-05
Number of iterations = 2 tolerance = 8.75223366004e-10
mean ddh =  -2.72103569683e-06
step = 32577  avg d_h= 0.0877703452236
Number of iterations = 1 tolerance = 2.37838954249e-05
Number of iterations = 2 tolerance = 8.68564276058e-10
mean ddh =  -2.72100681276e-06
step = 32578  avg d_h= 0.0877677865548
Number of iterations = 1 tolerance = 2.37848287411e-05
Number of iterations = 2 tolerance = 8.766631

Number of iterations = 1 tolerance = 2.38226156491e-05
Number of iterations = 2 tolerance = 8.74900818459e-10
mean ddh =  -2.71979188591e-06
step = 32620  avg d_h= 0.0876603470143
Number of iterations = 1 tolerance = 2.38235368845e-05
Number of iterations = 2 tolerance = 8.69989691399e-10
mean ddh =  -2.7197628795e-06
step = 32621  avg d_h= 0.0876577895152
Number of iterations = 1 tolerance = 2.38244685565e-05
Number of iterations = 2 tolerance = 8.63227461734e-10
mean ddh =  -2.71973386774e-06
step = 32622  avg d_h= 0.0876552320434
Number of iterations = 1 tolerance = 2.38253933038e-05
Number of iterations = 2 tolerance = 8.70997191038e-10
mean ddh =  -2.71970485021e-06
step = 32623  avg d_h= 0.0876526745989
Number of iterations = 1 tolerance = 2.38263045633e-05
Number of iterations = 2 tolerance = 8.68340871429e-10
mean ddh =  -2.71967583025e-06
step = 32624  avg d_h= 0.0876501171817
Number of iterations = 1 tolerance = 2.38272342572e-05
Number of iterations = 2 tolerance = 8.6575105

Number of iterations = 1 tolerance = 2.38650859213e-05
Number of iterations = 2 tolerance = 8.78738887211e-10
mean ddh =  -2.71845352436e-06
step = 32666  avg d_h= 0.0875427303505
Number of iterations = 1 tolerance = 2.38660123556e-05
Number of iterations = 2 tolerance = 8.67490551615e-10
mean ddh =  -2.71842435257e-06
step = 32667  avg d_h= 0.0875401741101
Number of iterations = 1 tolerance = 2.38669478575e-05
Number of iterations = 2 tolerance = 8.60686244497e-10
mean ddh =  -2.71839517721e-06
step = 32668  avg d_h= 0.0875376178971
Number of iterations = 1 tolerance = 2.38678668666e-05
Number of iterations = 2 tolerance = 8.6661006149e-10
mean ddh =  -2.71836599947e-06
step = 32669  avg d_h= 0.0875350617116
Number of iterations = 1 tolerance = 2.38687959203e-05
Number of iterations = 2 tolerance = 8.72787481176e-10
mean ddh =  -2.71833681772e-06
step = 32670  avg d_h= 0.0875325055535
Number of iterations = 1 tolerance = 2.38697005708e-05
Number of iterations = 2 tolerance = 8.9629856

Number of iterations = 1 tolerance = 2.39073875491e-05
Number of iterations = 2 tolerance = 2.31110214744e-09
mean ddh =  -2.71710916602e-06
step = 32712  avg d_h= 0.0874251717233
Number of iterations = 1 tolerance = 2.39083413815e-05
Number of iterations = 2 tolerance = 2.47755443349e-09
mean ddh =  -2.71707991286e-06
step = 32713  avg d_h= 0.0874226167471
Number of iterations = 1 tolerance = 2.39092898347e-05
Number of iterations = 2 tolerance = 2.66126009816e-09
mean ddh =  -2.71705065898e-06
step = 32714  avg d_h= 0.0874200617985
Number of iterations = 1 tolerance = 2.39102485551e-05
Number of iterations = 2 tolerance = 2.87165659513e-09
mean ddh =  -2.71702140511e-06
step = 32715  avg d_h= 0.0874175068774
Number of iterations = 1 tolerance = 2.3911208527e-05
Number of iterations = 2 tolerance = 3.11088645863e-09
mean ddh =  -2.71699215349e-06
step = 32716  avg d_h= 0.0874149519837
Number of iterations = 1 tolerance = 2.39121789844e-05
Number of iterations = 2 tolerance = 3.3857091

In [31]:
figure()
imshow_grid_at_node(mg, h)
title('head')

figure()
plot_links(mg, 'conduit__discharge')
title('discharge')


figure()
plot_links(mg, 'hydraulic__diameter')
title('d_H')


In [33]:
mc.run_one_step()
pfn.run_one_step()


mean ddh =  0.000919795601822
Number of iterations = 1 tolerance = 0.0111341793689
Number of iterations = 2 tolerance = 0.000130098183271


In [27]:
mg.calc_net_flux_at_node(Q, out=net_node_flux)
net_Q = mg.calc_net_flux_at_node(Q)/100. - mg.at_node['input__discharge'] 
figure()
imshow_grid_at_node(mg, net_Q)


Grid query results:

{'grid location': {'x_coord': 874.51298701298697,
                   'y_coord': 721.91558441558459},
 'node': {'ID': 114,
          'column': 9,
          'row': 7}}


In [20]:
mg.at_node['input__discharge']

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [24]:
from scipy.optimize import fsolve
def network_residuals(heads):
    h[mg.core_nodes] = heads
    mg.calc_grad_at_link(h, out=dhdx)
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    return mg.calc_net_flux_at_node(Q, out=net_node_flux)[mg.core_nodes]/mg.dx - mg.at_node['input__discharge'][mg.core_nodes]
#This function doesn't account for inflow at nodes


In [25]:
mc.run_one_step()
mg.calc_grad_at_link(h, out=dhdx)
mg.at_node['hydraulic__head'][mg.core_nodes] = fsolve(network_residuals, h[mg.core_nodes])
mg.at_link['conduit__discharge'][mg.core_nodes] = np.sign(dhdx[mg.core_nodes])*np.sqrt(np.fabs(dhdx[mg.core_nodes]))

print "mean h =", mean(mg.at_node['hydraulic__head'][mg.core_nodes] )
print "mean Q =", mean(Q)

mean ddh =  -5.66192997218e-18
mean h = 166.222550077
mean Q = 0.0865514465643


In [22]:
mg.dx

100.0

In [26]:
figure()
imshow_grid_at_node(mg, h)

In [24]:
figure()
plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
print mean(mg.at_link['hydraulic__diameter'])

3.50074727364e+69


In [56]:
figure()
plot_links(mg, 'conduit__discharge', autoscale=True, magnitude=True)
#print mean(mg.at_link['hydraulic__diameter'])

In [10]:
mc = MeltCreep(mg, dt=5000.)
#mean(mc.Q)

In [17]:
plot(mg.at_link['hydraulic__diameter'][200:300])


Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902


Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.6482677381

In [19]:
#figure()

pfn.run_one_step()
mc.run_one_step()

Number of iterations = 1 tolerance = 0.0031818002267
Number of iterations = 2 tolerance = 1.26436621244e-05
mean ddh =  -6.86035808489e-06


In [41]:
#for m in mc.melt:
#    print m
print max(mc.melt[~np.isnan(mc.melt)])
print max(mc.creep)
#print mc.d_h.mean()
#print mg.at_link['hydraulic__diameter']

3.35041761374e-06
2.12710591355e-06


In [15]:
#mc.run_one_step()

mc._grid.at_link['conduit__discharge']

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])